In [5]:
%load_ext autoreload
%autoreload 2

import smtplib as smtp 
import ssl 
import os

import queue
import multiprocessing

from functions import *
from master_functions import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Mail

In [78]:
mail = 'drainingytb@gmail.com'
pswd = 'YoDH@ck3r'

In [3]:
ssl_cntxt = ssl.create_default_context()

In [14]:
smtp.SMTP_SSL()


with smtplib.SMTP_SSL("smtp.gmail.com", port=465, context=ssl_cntxt) as server:
    server.login(mail, pswd)
    server.sendmail(from_addr=mail,
                    to_addrs='xabier.rubiato@epfl.ch',
                    msg='Finish Process')

## Download

In [3]:
from functions import yt_dl

### Vine playlist test 

In [2]:
URL = 'https://www.youtube.com/watch?v=2S7Yak00e6E&list=PLeu1bSfyWCDK4KgsF2pJybJkWXKPHgD6b'
PLAYLIST = 'test'
N_ITEMS = 6 
PTH_PLYLST = pth_plylst(name=PLAYLIST)

In [4]:
print('Start download of playlist : {}\n'.format(PLAYLIST))
yt_dl(url= URL,
      playlist=PLAYLIST,
      n_items= N_ITEMS)
print('Finish download of playlist : {}\n'.format(PLAYLIST))

Start download of playlist : test

[youtube:playlist] Downloading playlist PLeu1bSfyWCDK4KgsF2pJybJkWXKPHgD6b - add --no-playlist to just download video 2S7Yak00e6E
[youtube:playlist] PLeu1bSfyWCDK4KgsF2pJybJkWXKPHgD6b: Downloading webpage
[download] Downloading playlist: Alignments as vines
[youtube:playlist] playlist Alignments as vines: Downloading 6 videos
[download] Downloading video 1 of 6
[youtube] 2S7Yak00e6E: Downloading webpage
[youtube] 2S7Yak00e6E: Downloading video info webpage
[info] Writing video description metadata as JSON to: videos/test/2S7Yak00e6E/data/2S7Yak00e6E_1920x1080.info.json


[download] videos/test/2S7Yak00e6E/data/2S7Yak00e6E_1920x1080.mkv has already been downloaded and merged
[download] Downloading video 2 of 6
[youtube] qoqZ9T5O29g: Downloading webpage
[youtube] qoqZ9T5O29g: Downloading video info webpage
[info] Writing video description metadata as JSON to: videos/test/qoqZ9T5O29g/data/qoqZ9T5O29g_1920x1080.info.json


[download] videos/test/qoqZ9T5O29g/data/qoqZ9T5O29g_1920x1080.mkv has already been downloaded and merged
[download] Downloading video 3 of 6
[youtube] IMVmevu6mE4: Downloading webpage
[youtube] IMVmevu6mE4: Downloading embed webpage
[youtube] IMVmevu6mE4: Refetching age-gated info webpage
[info] Writing video description metadata as JSON to: videos/test/IMVmevu6mE4/data/IMVmevu6mE4_1920x1080.info.json
[download] videos/test/IMVmevu6mE4/data/IMVmevu6mE4_1920x1080.mp4 has already been downloaded and merged
[download] Downloading video 4 of 6
[youtube] MNMYYEdJoC0: Downloading webpage
[youtube] MNMYYEdJoC0: Downloading video info webpage
[info] Writing video description metadata as JSON to: videos/test/MNMYYEdJoC0/data/MNMYYEdJoC0_1920x1080.info.json
[download] videos/test/MNMYYEdJoC0/data/MNMYYEdJoC0_1920x1080.mp4 has already been downloaded and merged
[download] Downloading video 5 of 6
[youtube] -Ia18bqO-9w: Downloading webpage
[youtube] -Ia18bqO-9w: Downloading video info webpage
[inf

[download] videos/test/-Ia18bqO-9w/data/-Ia18bqO-9w_1920x1080.mkv has already been downloaded and merged
[download] Downloading video 6 of 6
[youtube] yd7d51AgSso: Downloading webpage
[youtube] yd7d51AgSso: Downloading video info webpage
[info] Writing video description metadata as JSON to: videos/test/yd7d51AgSso/data/yd7d51AgSso_1920x1080.info.json
[download] Destination: videos/test/yd7d51AgSso/data/yd7d51AgSso_1920x1080.f248.webm
[download] 100% of 43.28MiB in 00:1157MiB/s ETA 00:005
[download] Destination: videos/test/yd7d51AgSso/data/yd7d51AgSso_1920x1080.f251.webm
[download] 100% of 4.08MiB in 00:0168MiB/s ETA 00:004
[ffmpeg] Merging formats into "videos/test/yd7d51AgSso/data/yd7d51AgSso_1920x1080.webm"
Deleting original file videos/test/yd7d51AgSso/data/yd7d51AgSso_1920x1080.f248.webm (pass -k to keep)
Deleting original file videos/test/yd7d51AgSso/data/yd7d51AgSso_1920x1080.f251.webm (pass -k to keep)
[download] Finished downloading playlist: Alignments as vines
Finish downloa

### Download one video and put it into playlist 

In [5]:
URL = 'https://www.youtube.com/watch?v=PqyPW-Bdd4E'
PLYLST = 'test2'
yt_dl(URL, PLYLST)

[youtube] PqyPW-Bdd4E: Downloading webpage
[youtube] PqyPW-Bdd4E: Downloading video info webpage
[info] Writing video description metadata as JSON to: videos/test2/PqyPW-Bdd4E/data/PqyPW-Bdd4E_1920x1080.info.json


[download] videos/test2/PqyPW-Bdd4E/data/PqyPW-Bdd4E_1920x1080.mkv has already been downloaded and merged


## Multiprocessing

### Worker class and functions

In [2]:
%load_ext autoreload
%autoreload 2
import os
from multiprocessing import Process, Queue, cpu_count

from functions import pth_plylst, pth_vid
from master_functions import iter0, sfm_pipe

In [17]:
class Worker:

    def __init__(self, task_queue):
        self.q = task_queue


    def Qiter0(self, args):
        """Performs iter0() and enqueues an sfm_pipe process with the generated sets."""
        path_set_dir, width = iter0(*args)
        pth_sets = [os.path.join(path_set_dir, el) for el in os.listdir(path_set_dir)]

        # Enqueue next tasks
        for pth_set in pth_sets:
            args = (pth_set, width)
            self.q.put(('sfm_pipe', args))


    def sfm_pipe(self, pth, width):
        sfm_pipe(pth_set=pth, width=width)


    def work(self, func, args):

        if func == 'Qiter0':
            self.Qiter0(args)

        elif func == 'sfm_pipe':
            print('SFM PIPE')
            print(*args)
            self.sfm_pipe(*args)


def worker_exec(task_queue, wip_queue):
    # Work until no tasks are pending

    worker = Worker(task_queue)
    
    if not task_queue.empty():
        # instanciate worker
        func, args = task_queue.get()
        worker.work(func, args)

        

def drain(number_processes, rate, f_frms, f_ftrs, f_mtchs, sample,plylsts=[], vids=[]):
    # Enqueue Qiter0 tasks:
    Q = Queue()
    for plylst in plylsts:
        for v_id in os.listdir(pth_plylst(plylst)):
            args = (pth_vid(v_id, plylst),
                    rate, f_frms, f_ftrs, f_mtchs, sample)
            Q.put(('Qiter0', args))

    for v_id in vids:
        args = (pth_vid(v_id),
                rate, f_frms, f_ftrs, f_mtchs, sample)
        Q.put(('Qiter0', args))

    for i in range(number_processes):
        p = Process(target=worker_exec, args=(Q,))
        p.start()

In [ ]:
class Worker2:

    def __init__(self, task_queue):
        self.q = task_queue


    def Qiter0(self, args):
        """Performs iter0() and enqueues an sfm_pipe process with the generated sets."""
        path_set_dir, width = iter0(*args)
        pth_sets = [os.path.join(path_set_dir, el) for el in os.listdir(path_set_dir)]

        # Enqueue next tasks
        for pth_set in pth_sets:
            args = (pth_set, width)
            self.q.put(('sfm_pipe', args))


    def sfm_pipe(self, pth, width):
        sfm_pipe(pth_set=pth, width=width)


    def work(self, func, args):

        if func == 'Qiter0':
            self.Qiter0(args)

        elif func == 'sfm_pipe':
            print('SFM PIPE')
            print(*args)
            self.sfm_pipe(*args)


def worker_exec2(task_queue, wip_l):
    # Work until no tasks are pending
    
    if not task_queue.empty():
    worker = Worker2(task_queue)
        # instanciate worker
        func, args = task_queue.get()
        worker.work(func, args)


def drain2(number_processes, rate, f_frms, f_ftrs, f_mtchs, sample,
          plylsts=[], vids=[]):
    # Enqueue Qiter0 tasks:
    Q = Queue()
    wip_l = []
    
    for plylst in plylsts:
        for v_id in os.listdir(pth_plylst(plylst)):
            args = (pth_vid(v_id, plylst),
                    rate, f_frms, f_ftrs, f_mtchs, sample)
            Q.put(('Qiter0', args))

    for v_id in vids:
        args = (pth_vid(v_id),
                rate, f_frms, f_ftrs, f_mtchs, sample)
        Q.put(('Qiter0', args))

    while not (Q.empty() and len(wip_len) == 0): 
        
        p = Process(target=worker_exec2, args=(Q,wip_l))
        p.start()

In [1]:
l = []
l.append('a')
l.append('a')
l.append('a')
l.append('b')
l.append('a')
l.append('a')
l.append('a')

In [11]:
len(l)

0

### Launch processes: 

In [19]:
NUMBER_PROCESSES = cpu_count() 
PLYLST = ['SHIBUYA']
VIDS = []
f_frms  = f_ftrs = f_mtchs = True 
# iter0(path_vid, rate, sample=False, frame_force=False, feature_force=False, match_force=False):
drain(NUMBER_PROCESSES,vids=VIDS, plylsts=PLYLST, rate=1,
      f_frms=True,
      f_ftrs=True,
      f_mtchs=True, 
      sample=True)

Process Process-35:
Traceback (most recent call last):
  File "/home/dhacker/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/dhacker/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-17-c8b8fda4677f>", line 42, in worker_exec
    worker.work(func, args)
  File "<ipython-input-17-c8b8fda4677f>", line 25, in work
    self.Qiter0(args)
  File "<ipython-input-17-c8b8fda4677f>", line 9, in Qiter0
    path_set_dir, width = iter0(*args)
  File "/home/dhacker/Draining-Youtube/master_functions.py", line 70, in iter0
    path_sets =  make_sets(v_id, plylst)
  File "/home/dhacker/Draining-Youtube/master_functions.py", line 27, in make_sets
    path_frames=pth_frms(v_id, plylst)))
  File "/home/dhacker/Draining-Youtube/functions.py", line 434, in bin_matches_to_adja_mat
    return make_adj_mat(extract_matches(path_mtchs), path_frames)
  File "/home/dhack

## Tests

### Test iter 0 

In [4]:
from master_functions import iter0

In [5]:
path_set =  '/Users/xrubiato/DHLab/Draining-Youtube/videos/PqyPW-Bdd4E'

In [7]:
r = (pth_sets, width) =  iter0(path_vid=path_set)

print(r)

('/Users/xrubiato/DHLab/Draining-Youtube/videos/PqyPW-Bdd4E/sets', 1920)


### Test sfm_pipe

In [2]:
from master_functions import sfm_pipe

In [1]:
path_set = '/Users/xrubiato/DHLab/Draining-Youtube/videos/PqyPW-Bdd4E/sets/set_4'
width = 1920

In [3]:
sfm_pipe(path_set, width)